In [1]:
#| default_exp models.product_key.memory

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [4]:
#| export
import math, numpy as np, torch

from torch import nn
from typing import Optional
from dataclasses import dataclass
from torch.nn import functional as F

from xcai.core import store_attr

In [5]:
#| export
from torch.distributed._composable.fsdp import MixedPrecisionPolicy, fully_shard
from torch.distributed.tensor.parallel import parallelize_module

from xcai.models.product_key.colwise_embedding_bag import ColwiseEmbeddingBag, xFormerEmbeddingBag

## Load data

In [6]:
from xcai.main import *
from xcai.models.modeling_utils import Pooling

In [7]:
output_dir = '/scratch/scai/phd/aiz218323/outputs/mogicX/03_ngame-for-wikiseealsotitles'

data_dir = '/home/scai/phd/aiz218323/scratch/datasets/benchmarks/'
config_file = 'wikiseealsotitles'
config_key = 'data_meta'

mname = 'sentence-transformers/msmarco-distilbert-base-v4'

In [8]:
pkl_dir = '/scratch/scai/phd/aiz218323/datasets/processed/'

In [9]:
pkl_file = f'{pkl_dir}/mogicX/wikiseealsotitles_data-meta_distilbert-base-uncased_sxc.joblib'

In [10]:
block = build_block(pkl_file, config_file, True, config_key, data_dir=data_dir, n_sdata_meta_samples=3)

In [164]:
batch = block.train.one_batch(100)

In [165]:
batch.keys()

dict_keys(['data_idx', 'data_identifier', 'data_input_text', 'data_input_ids', 'data_attention_mask', 'plbl2data_idx', 'plbl2data_data2ptr', 'lbl2data_idx', 'lbl2data_data2ptr', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'lbl2data_attention_mask', 'pcat2data_idx', 'pcat2data_data2ptr', 'cat2data_idx', 'cat2data_data2ptr', 'cat2data_identifier', 'cat2data_input_text', 'cat2data_input_ids', 'cat2data_attention_mask', 'pcat2lbl_idx', 'pcat2lbl_lbl2ptr', 'cat2lbl_idx', 'cat2lbl_lbl2ptr', 'cat2lbl_identifier', 'cat2lbl_input_text', 'cat2lbl_input_ids', 'cat2lbl_attention_mask', 'cat2lbl_data2ptr', 'pcat2lbl_data2ptr'])

In [13]:
from transformers import DistilBertModel
m = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [14]:
o = m(input_ids=batch['data_input_ids'], attention_mask=batch['data_attention_mask'])
o = Pooling.mean_pooling(o.last_hidden_state, batch['data_attention_mask'])

In [68]:
o.shape

torch.Size([100, 768])

## `Query MLP`

In [69]:
#| export
class QueryMLP(nn.Module):
    
    def __init__(self, input_dim, heads, k_dim, sizes, bias=False, batchnorm=False):
        super().__init__()
        store_attr('input_dim,heads,k_dim,sizes')
        assert sizes[0] == input_dim
        assert sizes[-1] == (heads * k_dim)
        self.mlp = QueryMLP.get_mlp(list(sizes), bias=bias, batchnorm=batchnorm)

    @staticmethod
    def get_mlp(sizes, bias=True, batchnorm=True):
        """
        Generate a feedforward neural network.
        """
        assert len(sizes) >= 2
        pairs = [(sizes[i], sizes[i+1]) for i in range(len(sizes) - 1)]
        
        layers = []
        for i, (dim_in, dim_out) in enumerate(pairs):
            layers.append(nn.Linear(dim_in, dim_out, bias=bias))
            if batchnorm:
                layers.append(nn.BatchNorm1d(dim_out))
            if i < len(pairs) - 1:
                layers.append(nn.ReLU())

        return nn.Sequential(*layers)

    def forward(self, x):
        assert x.shape[-1] == self.input_dim
        x = x.contiguous().view(-1, self.input_dim) if x.dim() > 2 else x
        bs = len(x)
        
        o = self.mlp(x)

        assert o.shape == (bs, self.heads * self.k_dim)
        return o.view(bs * self.heads, self.k_dim)
        

### Example

In [63]:
m = QueryMLP(16, 2, 32, (16, 32, 64))

In [64]:
m.mlp

Sequential(
  (0): Linear(in_features=16, out_features=32, bias=False)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=64, bias=False)
)

In [65]:
x = torch.randn(10, 16)

In [66]:
o = m(x)

In [67]:
o.shape

torch.Size([20, 32])

## `Hashing Memory`

In [178]:
#| export
class HashingMemory(nn.Module):

    VALUES = None
    EVAL_MEMORY = True

    def __init__(
        self,
        input_dim:int,
        output_dim:int,
        
        value_fixed_lr:Optional[float] = 0.001,
        
        k_dim:Optional[int] = 512,
        v_dim:Optional[int] = -1,
        
        num_heads:Optional[int] = 4,
        num_knn:Optional[int] = 32,
        
        num_keys:Optional[int] = 1024,
        use_query_bias:Optional[bool] = True,
        use_query_batchnorm:Optional[bool] = False,
        
        input_dropout:Optional[float] = 0.0,
        query_dropout:Optional[float] = 0.0,
        value_dropout:Optional[float] = 0.0,

        use_gate:Optional[bool] = False,
        share_values:Optional[bool] = True,
        swilu_projection:Optional[bool] = True,
        use_peer_variant:Optional[bool] = False,  # Replaces the PK memory with the PEER variant (Parameter Efficient Expert Retrieval)
    ):
        # dropout
        assert 0 <= input_dropout < 1
        assert 0 <= query_dropout < 1
        assert 0 <= value_dropout < 1
        
        store_attr('input_dropout,query_dropout,value_dropout')
        
        assert k_dim >= 2
        assert k_dim % 2 == 0
        assert num_heads >= 2

        # PEER variant
        assert not (
            use_peer_variant and v_dim > 0
        ), f"Cannot use PEER variant with a value dimension different from the input dimension (v_dim=-1)"

        if use_query_batchnorm:
            logger.warning(
                "WARNING: If you use batch normalization, be sure that you use batches of sentences with the same size at training time. Otherwise, the padding token will result in incorrect mean/variance estimations in the BatchNorm layer."
            )

        # initialize
        super().__init__()
        
        store_attr('use_peer_variant,input_dim,output_dim,k_dim,num_heads,num_knn,share_values,value_fixed_lr')
        self.num_keys, self.size = num_keys, num_keys**2
        self.v_dim = v_dim if v_dim > 0 else output_dim

        # values initialization
        self.swilu_proj = swilu_projection
        self.v_proj = v_dim > 0 or self.swilu_proj

        # initialize keys
        self.keys = nn.Parameter(
            torch.empty(2 * self.num_heads * int(self.size**0.5), self.k_dim // 2)
        )
        
        self.original = not self.share_values or HashingMemory.VALUES is None
        
        # initialize the values
        if self.original:
            if not self.use_peer_variant:  # PK
                self.values = xFormerEmbeddingBag(self.size, self.v_dim)
                HashingMemory.VALUES = self.values
            else:  # PEER
                self.values_u = nn.Embedding(self.size, self.v_dim)
                self.values_v = nn.Embedding(self.size, self.v_dim)
                HashingMemory.VALUES = self.values_u, self.values_v
        else:
            if not self.use_peer_variant:  # PK
                self.values = None
            else:  # PEER
                self.values_u = None
                self.values_v = None

        if self.v_proj:
            proj_input = v_dim
            if self.swilu_proj and proj_input < 0: proj_input = output_dim
            self.value_proj = torch.nn.Linear(proj_input, output_dim)
            
        if self.swilu_proj: self.swilu_projection = torch.nn.Linear(self.input_dim, proj_input)
            
        # gated memory
        self.gating = torch.nn.Linear(input_dim, 1) if use_gate else None

        # query network
        l_sizes = (self.input_dim, self.num_heads * self.k_dim)

        self.query_proj = QueryMLP(
            self.input_dim,
            self.num_heads,
            self.k_dim,
            l_sizes,
            bias=use_query_bias,
            batchnorm=use_query_batchnorm,
        )

    def mp_parallelize(self, mesh, model_args, distributed_args, param_dtype):
        fsdp_config = dict(
            mp_policy=(
                MixedPrecisionPolicy(
                    param_dtype=param_dtype,
                    # reduce_dtype=torch.float32,
                    reduce_dtype=torch.bfloat16,
                )
            ),
            mesh=mesh["dp_replicate"],
        )
        # parallelize the module
        if distributed_args.memory_parallel_size > 1:
            assert (
                not self.use_peer_variant
            ), f"The PEER variant does not have a memory parallel implementation"
            if self.original:
                layer_plan = {"values": ColwiseEmbeddingBag()}
                parallelize_module(
                    self,
                    mesh["memory_parallel"],
                    layer_plan,
                )

        # share the parameters
        if self.original:
            if not self.use_peer_variant:
                self.values = fully_shard(
                    self.values, **fsdp_config, reshard_after_forward=False
                )
            else:
                self.values_u = fully_shard(
                    self.values_u, **fsdp_config, reshard_after_forward=False
                )
                self.values_v = fully_shard(
                    self.values_v, **fsdp_config, reshard_after_forward=False
                )
        if self.mem_share_values and self.original:
            if not self.use_peer_variant:
                HashingMemory.VALUES = self.values
            else:
                HashingMemory.VALUES = self.values_u, self.values_v
        if self.mem_share_values and not self.original:
            if not self.use_peer_variant:
                self.values = HashingMemory.VALUES
            else:
                self.values_u, self.values_v = HashingMemory.VALUES

    def reset_parameters(self, init_std=None, factor=1.0):
        # keys
        bound = 1 / math.sqrt(self.k_dim)
        nn.init.uniform_(self.keys, a=-bound, b=bound)
        
        # values
        if self.original:
            if not self.use_peer_variant:
                nn.init.normal_(self.values.weight, mean=0, std=self.v_dim**-0.5)
            else:
                nn.init.normal_(self.values_u.weight, mean=0, std=self.v_dim**-0.5)
                nn.init.normal_(self.values_v.weight, mean=0, std=self.v_dim**-0.5)
                
        # queries
        nn.init.xavier_uniform_(self.query_proj.query_mlps[0].weight)
        
        # value projection
        if self.v_proj:
            nn.init.normal_(self.value_proj.weight, mean=0, std=self.output_dim**-0.5)
        if self.swilu_proj:
            nn.init.normal_(
                self.swilu_projection.weight, mean=0, std=self.output_dim**-0.5
            )
            
        # fixed learning rate:
        if self.original:
            if self.use_peer_variant:
                for p in self.values_u.parameters():
                    p.fixed_lr = self.value_fixed_lr
                    p.pk_value_param = True
                for p in self.values_v.parameters():
                    p.fixed_lr = self.value_fixed_lr
                    p.pk_value_param = True
            else:
                for p in self.values.parameters():
                    p.fixed_lr = self.value_fixed_lr
                    p.pk_value_param = True
                    
        if self.gating is not None:
            nn.init.normal_(self.gating.weight, mean=0, std=self.input_dim**-0.5)

    def get_indices(self, query, knn):
        assert query.dim() == 2 and query.size(1) == self.k_dim
        bs = len(query) // self.num_heads
        query = query.view(-1, self.num_heads, self.k_dim)
        half = self.k_dim // 2
        
        # keys : (heads, 2, n_keys, half)
        # keys1 : (heads, n_keys, half)
        keys = self.keys.view(self.num_heads, 2, -1, half)
        keys1 = keys[:, 0, :, :]
        keys2 = keys[:, 1, :, :]
        n_keys = len(keys[0][0])

        # split query for product quantization
        q1 = query[:, :, :half]  # (bs, heads, half)
        q2 = query[:, :, half:]  # (bs, heads, half)

        # compute indices with associated scores
        scores1 = torch.einsum(
            "blh, lkh->blk", q1, keys1
        )  # (bs , heads, n_keys ** 0.5)
        scores2 = torch.einsum(
            "blh, lkh->blk", q2, keys2
        )  # (bs , heads, n_keys ** 0.5)

        scores1, indices1 = scores1.topk(knn, dim=2, largest=True)  # (bs, heads, knn)
        scores2, indices2 = scores2.topk(knn, dim=2, largest=True)  # (bs, heads, knn)

        # cartesian product on best candidate keys
        all_scores = (
            scores1.view(bs, self.num_heads, knn, 1).expand(bs, self.num_heads, knn, knn)
            + scores2.view(bs, self.num_heads, 1, knn).expand(bs, self.num_heads, knn, knn)
        ).view(
            bs, self.num_heads, -1
        )  # (bs, heads, knn ** 2)
        all_indices = (
            indices1.view(bs, self.num_heads, knn, 1).expand(bs, self.num_heads, knn, knn)
            * n_keys
            + indices2.view(bs, self.num_heads, 1, knn).expand(bs, self.num_heads, knn, knn)
        ).view(
            bs, self.num_heads, -1
        )  # (bs, heads, knn ** 2)

        # select overall best scores and indices
        scores, best_indices = torch.topk(
            all_scores, k=knn, dim=2, largest=True, sorted=True
        )  # (bs, heads, knn)
        indices = all_indices.gather(2, best_indices)  # (bs, heads, knn)

        # return scores with indices
        assert scores.shape == indices.shape == (bs, self.num_heads, knn)
        return scores.view(bs * self.num_heads, knn), indices.view(bs * self.num_heads, knn)

    def get_scores(self, query, indices):
        assert query.dim() == 2 and query.size(1) == self.k_dim
        bs = len(query) // self.num_heads
        query = query.view(-1, self.num_heads, self.k_dim)
        half = self.k_dim // 2
        
        # keys : (heads, 2, n_keys, half)
        # keys1 : (heads, n_keys, half)
        keys = self.keys.view(self.num_heads, 2, -1, half)
        keys1 = keys[:, 0, :, :]
        keys2 = keys[:, 1, :, :]
        n_keys = len(keys[0][0])

        # split query for product quantization
        q1 = query[:, :, :half]  # (bs, heads, half)
        q2 = query[:, :, half:]  # (bs, heads, half)

        # compute indices with associated scores
        scores1 = torch.einsum(
            "blh, lkh->blk", q1, keys1
        )  # (bs , heads, num_keys ** 2)
        scores2 = torch.einsum(
            "blh, lkh->blk", q2, keys2
        )  # (bs , heads, num_keys ** 2)

        all_scores = (
            scores1.view(bs, self.num_heads, self.num_keys, 1).expand(bs, self.num_heads, self.num_keys, self.num_keys)
            + scores2.view(bs, self.num_heads, 1, self.num_keys).expand(bs, self.num_heads, self.num_keys, self.num_keys)
        ).view(
            bs, self.num_heads, -1
        )  # (bs, heads, num_keys ** 2)

        num_items = indices.size(1)
        indices = indices.unsqueeze(1).expand(-1, all_scores.size(1), -1)
        scores = all_scores.gather(dim=2, index=indices)

        assert scores.shape == indices.shape == (bs, self.num_heads, num_items)
        return scores.view(bs * self.num_heads, num_items), indices.contiguous().view(bs * self.num_heads, num_items)
        
    def forward(self, x, input_indices=None, use_self_linker=True, return_scores=False):
        """
        Read from the memory.
        """
        B, C = x.shape
        x = x.view(-1, self.input_dim)
        
        assert x.shape[-1] == self.input_dim
        prefix_shape = x.shape[:-1]

        bs = np.prod(prefix_shape)
        x = F.dropout(x, p=self.input_dropout, training=self.training)
        query = self.query_proj(x)  # (bs * heads, k_dim)
        query = F.dropout(query, p=self.query_dropout, training=self.training)  # (bs * heads, k_dim)
        assert query.shape == (bs * self.num_heads, self.k_dim)

        scores, indices = None, None
        
        # get scores
        if input_indices is not None: 
            scores, indices = self.get_scores(query, input_indices) # (bs * heads, num_items)
        
        # get indices
        if use_self_linker:
            sc, idx = self.get_indices(query, self.num_knn)  # (bs * heads, knn)
            scores = sc if scores is None else torch.cat([scores, sc], dim=1) # (bs * heads, knn + num_items)
            indices = idx if indices is None else torch.cat([indices, idx], dim=1) # (bs * heads, knn + num_items)

        num_items = indices.size(1)
        
        # store indices / scores (eval mode only - for usage statistics)
        if not self.training and HashingMemory.EVAL_MEMORY:
            self.last_indices = indices.view(bs, self.num_heads, num_items).detach().cpu()
            self.last_scores = scores.view(bs, self.num_heads, num_items).detach().cpu().float()

        # re-scoring
        scores = F.softmax(scores.float(), dim=-1).type_as(scores)  # (bs * heads, num_items)

        # merge heads / knn (since we sum heads)
        indices = indices.view(bs, self.num_heads * num_items)  # (bs, heads * num_items)
        scores = scores.view(bs, self.num_heads * num_items)  # (bs, heads * num_items)

        if not self.use_peer_variant:
            output = self.values(indices, scores)  # (bs, v_dim)
            if self.v_proj and not self.swilu_proj:
                output = self.value_proj(output)
            if self.swilu_proj:
                output = self.value_proj(output * F.silu(self.swilu_projection(input)))
        else:
            u = self.values_u(indices)
            x = torch.einsum(
                "bh, blh->bl", x, u
            )  # (bs, v_dim) , (bs, heads * knn, v_dim) -> (bs, heads * knn)
            x = F.gelu(x)  # This can be either GeLU or ReLU
            v = self.values_v(indices)
            x = x * scores  # (bs, heads * num_items)
            output = torch.einsum(
                "bl, blh->bh", x, v
            )  # (bs, heads * num_items) , (bs, heads * num_items, v_dim) -> (bs, v_dim)

        output = F.dropout(
            output, p=self.value_dropout, training=self.training
        )  # (bs, v_dim)

        # reshape output
        if len(prefix_shape) >= 2:
            output = output.view(prefix_shape + (self.v_dim,))  # (..., v_dim)

        if self.gating:
            output = F.sigmoid(self.gating(input)) * output
        output = output.view(B, C)
        
        return (output, scores, indices) if return_scores else output
        

### Example

In [179]:
from xcai.models.cachew import BaseMemory

In [180]:
num_keys = int(np.ceil(np.sqrt(block.train.dset.meta['cat_meta'].n_meta))); num_keys

810

In [181]:
m = HashingMemory(768, 768, num_keys=810, share_values=False, use_peer_variant=True)

In [182]:
input_indices = BaseMemory.align_indices(batch['cat2data_idx'], batch['cat2data_data2ptr'])

In [183]:
output = m(o, input_indices=input_indices, use_self_linker=True, return_scores=True)

In [184]:
output[0].shape

torch.Size([100, 768])